In [1]:
# the library used
import numpy as np
import pandas as pd
import pysubgroup as ps
import re
from sklearn.feature_extraction import DictVectorizer
import time
import ast
import string 
import imblearn
from sklearn.utils import resample
from sklearn.model_selection import train_test_split

## Helper Function

In [2]:
def tags_preprocess(tags):
    """
    input: tag string
    output: list of individual tags in the given tag string
    function: preprocess a single tag string 
    """
    tags = tags.replace("'","")
    tags = tags.replace(" ","")
    tags = tags.replace("[","")
    tags = tags.replace("]","")
    tags = tags.split(",")
    tags = [x.lower() for x in tags]
    return tags

In [3]:
def ingredients_preprocess(dataframe):
    """
    input: dataframe
    output: list of distinct ingredient list
    function: preprocess the ingredient columns and return a list of distinct ingredients
    """
    distinct_ingredients = []
    
    for i in range(len(dataframe)):
        ingredients = dataframe.iloc[i]['ingredient']
        
        r = re.compile('[A-Z]{1}[a-zA-Z]+')
        ingredients = str(ingredients)
        #ingredients = ''.join(i for i in ingredients if not i.isdigit())
        ingredients = ingredients.replace("'","")
        ingredients = ingredients.replace(" ","")
        ingredients = ingredients.replace("[","")
        ingredients = ingredients.replace("]","")
        # remove text inside parentheses
        ingredients = re.sub(r'\([^())]*\)',"", ingredients)
        ingredients = ingredients.split(",")
        ingredients = list(filter(r.match, ingredients))
        ingredients = [x.lower() for x in ingredients]
        distinct_ingredients += ingredients
        dataframe.iloc[i]['ingredient'] = ingredients
        
    return list(set(distinct_ingredients))

In [4]:
def get_recipe_countries(countries, data):
    """
    input: list of countries, dataframe
    output: selected dataframe whose recipes is from these countries
    function: select the rows in dataframe whose "tag" value contain one country tag
    """
    # add a new column class 
    drop_index = []
    for i in range(len(data)):
        tags = data.loc[i]["tags"]
        tags = tags_preprocess(tags)
        
        country_same =[l for l in countries if l in tags]
            
        if len(country_same) == 1:
            data.at[i, 'label'] = country_same[0]
        if len(country_same) == 0:
            drop_index.append(i)
        if len(country_same) > 1:
            drop_index.append(i)
            #data.at[i, 'label'] = 'overlap'
            
    # drop the columns which has no season tags
    data = data.drop(data.index[drop_index])
    return data

In [5]:
def convert_to_dict(arr):
    """
    Helper function to convect an array of ingredients to a dictionary
    """
    d={}
    for a in arr:
        d[a]=1
    return d

In [6]:
# extract comment user dataset from original dataset
def extract_com_user(data):
    """
    input: dataframe
    output: a new dataframe with all the comment user information
    function: spilt the dictionary of the column 'comment user' in original dataset
    """
    df_com = pd.DataFrame()
    for index, item in data['comment_user'].iteritems():
        if (item != '[]'):
            if (item != 'no comment'):
                array = ast.literal_eval(item)
                df_array = pd.DataFrame(array)
                df_array['recipe_id'] = index
                df_com = pd.concat([df_com,df_array])
    return df_com

In [7]:
def sub_cat_in_com(data):
    """
    input: dataframe
    output: a new dataframe with multi colunms
    function: add one subcategory of recipes to comment user dataset 
    """    
    punct = set(string.punctuation) 
    list_sub_cat = []
    
    df_com2 = pd.DataFrame()
    for index, item in data['calorie_value'].iteritems(): 
        if (item != None):
            list_sub = list(item)
            list_sub = ''.join(x for x in list_sub if x not in punct)
            list_sub_cat.append(list_sub)
    df_sub_cat = pd.DataFrame(list_sub_cat)
    df_sub_cat['calorie_value'] = df_sub_cat
    
    df_sub_cat['recipe_id'] = data['calorie_value'].index     
    df_com2 = pd.concat([df_com2,df_sub_cat])
    return df_com2

In [124]:
def add_recipe_info(data):
    """
    input: dataframe
    output: a new dataframe with multi colunms
    function: add one subcategory of recipes to comment user dataset 
    """    
    punct = set(string.punctuation) 
    # add recipe name in comment users data
    list_recipe_name = []
    df_recipe = pd.DataFrame()
    for index, item in data['recipe_name'].iteritems(): 
        if (item != None):
            list_name = list(item)
            list_name = ''.join(x for x in list_name if x not in punct)
            list_recipe_name.append(list_name)

    df_name = pd.DataFrame(list_recipe_name)
    df_name['recipe_id'] = data['recipe_name'].index 
    df_name = df_name.set_index(["recipe_id"])
    df_name
    df_recipe['recipe_name'] = df_name[0]

    # add recipe difficulty in comment users data
    list_recipe_diff = []
    df_recipe_diff = pd.DataFrame()
    for index, item in data['difficulty'].iteritems(): 
        if (item != None):
            list_diff = list(item)      
            list_diff = ''.join(x for x in list_diff if x not in punct)   
            list_recipe_diff.append(list_diff)

    df_diff = pd.DataFrame(list_recipe_diff)
    df_diff['recipe_id'] = data['difficulty'].index 
    df_diff = df_diff.set_index(["recipe_id"])
    df_recipe['difficulty'] = df_diff[0]

    # add recipe preparation_time in comment users data
    list_recipe_pre = []
    df_recipe_pre = pd.DataFrame()
    for index, item in data['preparation_time'].iteritems(): 
        if (item != None):
            list_pre = list(item)

            list_pre = ''.join(x for x in list_pre if x not in punct)

            list_recipe_pre.append(list_pre)

    df_pre = pd.DataFrame(list_recipe_pre)
    df_pre['recipe_id'] = data['preparation_time'].index 
    df_pre = df_pre.set_index(["recipe_id"])
    df_recipe['preparation_time'] = df_pre[0]

    # add recipe tags in comment users data
    list_recipe_tags = []
    df_recipe_tags = pd.DataFrame()
    for index, item in data['tags'].iteritems(): 
        if (item != None):
            list_tags = list(item)      
            list_tags = ''.join(x for x in list_tags if x not in punct)   
            list_recipe_tags.append(list_tags)


    df_tags = pd.DataFrame(list_recipe_tags)
    df_tags['recipe_id'] = data['tags'].index 
    df_tags = df_tags.set_index(["recipe_id"])
    df_recipe['tags'] = df_tags[0]

    # add recipe ingredient in comment users data
    list_recipe_ingredient = []
    df_recipe_ingredient = pd.DataFrame()
    for index, item in data['ingredient'].iteritems(): 
        if (item != None):
            list_ingredient = list(item) 
            list_ingredient = ''.join(x for x in list_ingredient if x not in punct)
            list_recipe_ingredient.append(list_ingredient)

    df_recipe_ingredient = pd.DataFrame(list_recipe_ingredient)
    df_recipe_ingredient['recipe_id'] = data['ingredient'].index 
    df_recipe_ingredient = df_recipe_ingredient.set_index(["recipe_id"])
    df_recipe['ingredient'] = df_recipe_ingredient[0]
    return df_recipe

In [9]:
def age_group(age):   
    """
    input: age value
    output: group description
    function: divide age value uinto 5 groups 
    """   
    bucket = None
    age = int(age)    
    if age < 30:
        bucket = '<30 Jahre'    
    if age in range(30, 41):
        bucket = '30-40 Jahre'        
    if age in range(40, 51):
        bucket = '40-50 Jahre'        
    if age in range(50, 61):
        bucket = '50-60 Jahre'
    if age >= 61:
        bucket = '60+ Jahre'
    return bucket

In [10]:
def calorie_level(calorie):   
    """
    input: calorie value
    output: group description
    function: divide calorie value into 3 groups 
    """   
    bucket = None
    calorie = int(calorie)    
    if calorie < 300:
        bucket = 'low_calorie'    
    if calorie in range(300, 500):
        bucket = 'medium_calorie'        
    if calorie >= 500:
        bucket = 'high_calorie'      
    return bucket

In [11]:
def remove_None(data, name):
    """
    Helper function to remove None value in one column
    """ 
    y = data[data[name] == 'None']
    index_n = y.index.tolist()
    data = data.drop(index = index_n)
    return data

In [12]:
def add_target(data, calorie_level):
    df_sub_group[country] = df_dum_car[calorie_level]
    return df_sub_group

## Subgroup Discovery

In this section we will use subgroup discovery to explore the association rules between attributes

- why we choose subgroup discovery?

because we find out that subgroup discovery is quite powerful compared to other data mining techniques. As long as we set differnt target with different search space, we can use use subgroup discovery technique to dig almost all interesting pattern that we want to explore from the dataset. 


In [133]:
# read the data
data = pd.read_csv("all_data.csv")


In [134]:
# extract the recipe id from recipe urls
list_cat_no = []
i = 0
for item in data['recipe_url']:
    list_cat_no.append(item.split('/')[4])
    
# add one column "recipe_id" into the dataset and set it as the index of dataset
data['recipe_id'] = list_cat_no
data = data.set_index(["recipe_id"])

In [135]:
data.head()

,categorize,recipe_name,tags,avg_score,difficulty,ingredient,rating_count,calorie,preparation_time,comment_user,recipe_url
recipe_id,,,,,,,,,,,
2369421376028274,Menüart,"""A bis Z""-Salat","['Salat', 'Gemüse', 'Sommer', 'Vegetarisch', 'Beilage', 'Snack', 'Früchte', 'Frucht']",2.67,simpel,"['1m.-große', 'Zucchini', '1', 'Apfel,säuerlich', '1', 'Zwiebel(n),rot', '2TL', 'Salz', '2EL', '...",1.0,NaN,15 Min,"[{'rating': 'rating-3', 'comment_time': '24.09.2015 17:22', 'name': 'Karikie', 'sex': 'weiblich'...",https://www.chefkoch.de/rezepte/2369421376028274/A-bis-Z-Salat.html\n
1883831306402031,Menüart,"""Aufgehende Sonne""","['Asien', 'Hauptspeise', 'Nudeln', 'Beilage', 'Geflügel', 'Wok', 'Thailand']",3.60,normal,"['500g', 'Nudeln(Mie-),ReisnudelnoderReis', '400g', 'Geflügelfleisch,geschnetzelt', '1', 'Ananas...",3.0,500kcal,30 Min,"[{'rating': 'rating-5', 'comment_time': '10.08.2017 14:56', 'name': 'Haubndaucher', 'sex': 'männ...",https://www.chefkoch.de/rezepte/1883831306402031/Aufgehende-Sonne.html\n
3615831543679585,Regional,"""Black and White"" New York Cheesecake","['Backen', 'USAoderKanada', 'Kuchen']",0.00,normal,"['250g', 'Bitterschokolade,70%', '300g', 'Schlagsahne']",0.0,NaN,60 Min,NaN,https://www.chefkoch.de/rezepte/3615831543679585/Black-and-White-New-York-Cheesecake.html\n
2046401331198519,Menüart,"""Bleib gesund""-Smoothie SuperNova","['Getränk', 'Vegetarisch', 'Frühstück', 'kalorienarm', 'Vegan', 'Frucht', 'Shake', 'Paleo']",4.03,simpel,"['1m.-große', 'Karotte(n)', '1', 'Apfel', '1Stück(e)', 'Ingwerwurzel,frisch,haselnussgroß', '1',...",84.0,205kcal,5 Min,"[{'rating': 'rating-1', 'comment_time': '13.01.2020 21:05', 'name': 't1n4a', 'sex': 'weiblich', ...",https://www.chefkoch.de/rezepte/2046401331198519/Bleib-gesund-Smoothie-SuperNova.html\n
3387501504018214,Menüart,"""Bottermelk Anballerste""","['Hauptspeise', 'Europa', 'Suppe', 'gebunden', 'Schnell', 'einfach', 'Schwein', 'Deutschland', '...",3.60,normal,"['500g', 'Kartoffel(n)', '1', 'Zwiebel(n)', '1EL', 'Butter', '2EL', 'Mehl', '1Liter', 'Buttermil...",3.0,NaN,15 Min,"[{'rating': 'rating-3', 'comment_time': '10.11.2017 18:59', 'name': 'Karikie', 'sex': 'weiblich'...",https://www.chefkoch.de/rezepte/3387501504018214/Bottermelk-Anballerste.html\n


## Association rules between comment user information and recipe attributes

- data imbalance: gender
- improve the quality of data (preprossing)
- sampling 
- improne quality of result: different search algorithmn and selectors choices
- if the quality is also very low, plot the t-SNE

- associations with "calorie" and comment user information

since the job has more than half non values, so here we only explore the the association between  marriage_status, gender, and age with recipe calorie

In [140]:
# extract the numerical value string in the column calorie 
pat = r"([-+]?\d*\.\d+|\d+)"
data["calorie_value"] = data["calorie"].str.extract(pat, flags=0, expand=True)

# drop all rows with nan value in both columns comment_user and calorie_value
data_com = data.dropna(subset=["comment_user",'calorie_value'])
len(data_com)


19058

In [142]:
# expand the corresponding recipe data for each comment_user
data_com = data_com[0:500]
# data_ingrent = ingredients_preprocess(data_com)
# data_ingrent
df_com_1 = extract_com_user(data_com)
df_com_2 = sub_cat_in_com(data_com)
df_com_3 = add_recipe_info(data_com)
df_com_new = df_com_1.merge(df_com_2, on='recipe_id', how='left')
df_com_new = df_com_new.merge(df_com_3, on='recipe_id', how='left')

df_com_new = df_com_new[['recipe_id','recipe_name','tags','difficulty','preparation_time','ingredient','name','rating','sex','age','marriage_status','comment_time','calorie_value']]
df_com_new = df_com_new.set_index(["recipe_id"])
df_com_new.head()

,recipe_name,tags,difficulty,preparation_time,ingredient,name,rating,sex,age,marriage_status,comment_time,calorie_value
recipe_id,,,,,,,,,,,,
1883831306402031,Aufgehende Sonne,Asien Hauptspeise Nudeln Beilage Geflügel Wok Thailand,normal,30 Min,500g NudelnMieReisnudelnoderReis 400g Geflügelfleischgeschnetzelt 1 Ananas 2 OrangenoderBlutoran...,Haubndaucher,rating-5,männlich,46 Jahre,Verheiratet,10.08.2017 14:56,500
1883831306402031,Aufgehende Sonne,Asien Hauptspeise Nudeln Beilage Geflügel Wok Thailand,normal,30 Min,500g NudelnMieReisnudelnoderReis 400g Geflügelfleischgeschnetzelt 1 Ananas 2 OrangenoderBlutoran...,Lumakath,rating-4,weiblich,49 Jahre,Verheiratet,19.01.2013 13:19,500
1883831306402031,Aufgehende Sonne,Asien Hauptspeise Nudeln Beilage Geflügel Wok Thailand,normal,30 Min,500g NudelnMieReisnudelnoderReis 400g Geflügelfleischgeschnetzelt 1 Ananas 2 OrangenoderBlutoran...,esther76,rating-4,weiblich,None,Verheiratet,21.10.2012 17:56,500
2046401331198519,Bleib gesundSmoothie SuperNova,Getränk Vegetarisch Frühstück kalorienarm Vegan Frucht Shake Paleo,simpel,5 Min,1mgroße Karotten 1 Apfel 1Stücke Ingwerwurzelfrischhaselnussgroß 1 OrangenSaftdavon ½ ZitronenSa...,t1n4a,rating-1,weiblich,25 Jahre,None,13.01.2020 21:05,205
2046401331198519,Bleib gesundSmoothie SuperNova,Getränk Vegetarisch Frühstück kalorienarm Vegan Frucht Shake Paleo,simpel,5 Min,1mgroße Karotten 1 Apfel 1Stücke Ingwerwurzelfrischhaselnussgroß 1 OrangenSaftdavon ½ ZitronenSa...,patty89,rating-4,weiblich,None,Verheiratet,03.09.2019 13:18,205


In [143]:
# remove none value in the whole data set 
df_com_no_none = df_com_new.mask(df_com_new.astype(object).eq('None')).dropna()
df_com_new = df_com_no_none

df_com_new = df_com_new.reset_index()
df_com_new

,recipe_id,recipe_name,tags,difficulty,preparation_time,ingredient,name,rating,sex,age,marriage_status,comment_time,calorie_value
0,1883831306402031,Aufgehende Sonne,Asien Hauptspeise Nudeln Beilage Geflügel Wok Thailand,normal,30 Min,500g NudelnMieReisnudelnoderReis 400g Geflügelfleischgeschnetzelt 1 Ananas 2 OrangenoderBlutoran...,Haubndaucher,rating-5,männlich,46 Jahre,Verheiratet,10.08.2017 14:56,500
1,1883831306402031,Aufgehende Sonne,Asien Hauptspeise Nudeln Beilage Geflügel Wok Thailand,normal,30 Min,500g NudelnMieReisnudelnoderReis 400g Geflügelfleischgeschnetzelt 1 Ananas 2 OrangenoderBlutoran...,Lumakath,rating-4,weiblich,49 Jahre,Verheiratet,19.01.2013 13:19,500
2,2046401331198519,Bleib gesundSmoothie SuperNova,Getränk Vegetarisch Frühstück kalorienarm Vegan Frucht Shake Paleo,simpel,5 Min,1mgroße Karotten 1 Apfel 1Stücke Ingwerwurzelfrischhaselnussgroß 1 OrangenSaftdavon ½ ZitronenSa...,mackogreen,rating-5,weiblich,53 Jahre,Verheiratet,18.12.2017 07:03,205
3,2046401331198519,Bleib gesundSmoothie SuperNova,Getränk Vegetarisch Frühstück kalorienarm Vegan Frucht Shake Paleo,simpel,5 Min,1mgroße Karotten 1 Apfel 1Stücke Ingwerwurzelfrischhaselnussgroß 1 OrangenSaftdavon ½ ZitronenSa...,Tweetschekätzchen,rating-4,weiblich,28 Jahre,Vergeben,12.04.2017 09:55,205
4,2046401331198519,Bleib gesundSmoothie SuperNova,Getränk Vegetarisch Frühstück kalorienarm Vegan Frucht Shake Paleo,simpel,5 Min,1mgroße Karotten 1 Apfel 1Stücke Ingwerwurzelfrischhaselnussgroß 1 OrangenSaftdavon ½ ZitronenSa...,BlueLili,rating-5,weiblich,29 Jahre,Vergeben,11.06.2016 09:54,205
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5120,1882491306235348,ApfelBeignets,Vegetarisch Schnell einfach Fingerfood Dessert Frittieren Gluten Lactose,simpel,40 Min,4große ÄpfelBoskop 2EL Zitronensaft 2EL Rum 50g Zucker 1Prisen Salz 2 Eier 2EL Öl 200g Mehl 250m...,kleinehobbits,rating-5,weiblich,45 Jahre,Verheiratet,18.03.2014 22:03,1626
5121,1882491306235348,ApfelBeignets,Vegetarisch Schnell einfach Fingerfood Dessert Frittieren Gluten Lactose,simpel,40 Min,4große ÄpfelBoskop 2EL Zitronensaft 2EL Rum 50g Zucker 1Prisen Salz 2 Eier 2EL Öl 200g Mehl 250m...,P1963,rating-4,weiblich,57 Jahre,Vergeben,22.11.2012 09:49,1626
5122,1882491306235348,ApfelBeignets,Vegetarisch Schnell einfach Fingerfood Dessert Frittieren Gluten Lactose,simpel,40 Min,4große ÄpfelBoskop 2EL Zitronensaft 2EL Rum 50g Zucker 1Prisen Salz 2 Eier 2EL Öl 200g Mehl 250m...,Patissier1992,rating-4,männlich,27 Jahre,Vergeben,25.05.2011 11:30,1626
5123,3317061492672434,ApfelBirnenSellerieSalat,Salat Gemüse Vegetarisch Beilage Früchte Blanchieren,simpel,25 Min,1 Apfel 1 Birnen 50g Knollensellerie Salzwasser,Tombstone5,rating-5,weiblich,55 Jahre,Verheiratet,19.11.2018 13:14,325


In [62]:
df_com_new['marriage_status'].value_counts()

Verheiratet    24851
Vergeben       16914
Single          5299
Verwitwet       1418
Geschieden       331
Name: marriage_status, dtype: int64

In [63]:
# add calorie level and age group columns in the comment user information
df_com_new['calorie_level'] = df_com_new['calorie_value'].apply(calorie_level)
df_dum_car = pd.get_dummies(df_com_new['calorie_level'])
df_com_age = df_com_new
df_com_age["age_value"] = df_com_age["age"].str.extract(pat, flags=0, expand=True)
df_com_age['age_group'] = df_com_age['age_value'].apply(age_group)
df_com_car = df_com_new.join(df_dum_car, how='left')

In [80]:
import pysubgroup as ps
data = df_com_car[['sex', 'age_group', 'high_calorie', 'low_calorie', 'medium_calorie','marriage_status', 'job']]
target = ps.BinaryTarget('low_calorie', True)
searchspace = ps.create_selectors(data, ignore=['high_calorie', 'low_calorie', 'medium_calorie'])
task = ps.SubgroupDiscoveryTask (
    data, 
    target, 
    searchspace, 
    result_set_size=5, 
    depth=2, 
    qf=ps.WRAccQF())
result = ps.BeamSearch().execute(task)
pd.set_option('max_colwidth',100)
print(result.to_dataframe())

    quality                                       description
0  0.003987        age_group=='60+ Jahre' AND sex=='weiblich'
1  0.003526                            age_group=='60+ Jahre'
2  0.003071  marriage_status=='Verwitwet' AND sex=='weiblich'
3  0.002932                                   sex=='weiblich'
4  0.002867      age_group=='50-60 Jahre' AND sex=='weiblich'


In [ ]:
df_com_new['sex'].value_counts()

#### Deal with imbalanced data

"weiblich": "männlich" ratio is around 5:1, the data is quite imblanced with regard to the sex, we want to try to resample the dataset to help improve the data quality. We want to see if we can improve the quality of subgroup discovery

- upsampling

In [81]:
df_sampling = df_com_age[['rating','sex','age_group','marriage_status','calorie_level','calorie_value']]

# oversampling
# Separate input features and target
y = df_sampling['calorie_level']
X = df_sampling.drop('calorie_level', axis=1)

# setting up testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)

# concatenate our training data back together
X = pd.concat([X_train, y_train], axis=1)

# separate minority and majority classes
female = df_sampling[df_sampling['sex'] == 'weiblich']
male = df_sampling[df_sampling['sex'] == 'männlich']

# upsample minority
male_upsampled = resample(male,
                          replace=True, # sample with replacement
                          n_samples=len(female), # match number in majority class
                          random_state=27) # reproducible results

# combine majority and upsampled minority
upsampled = pd.concat([female, male_upsampled])
upsampled.reset_index(inplace = True)
# upsampled['sex'].value_counts()
#upsampled.head()

In [82]:
# add the dummyset into the upsampled dataset
upsampled_dum = pd.get_dummies(upsampled['calorie_level'])
upsampled_dum.reset_index(inplace = True)
upsampled_dum = upsampled_dum.drop('index', axis=1)

# prepare for the binary target columns
upsampled = upsampled.drop('index', axis=1)
upsampled_new = upsampled.join(upsampled_dum, how='left')
# upsampled_new = upsampled_new.drop('calorie_level',1)
upsampled_new

,rating,sex,age_group,marriage_status,calorie_level,calorie_value,high_calorie,low_calorie,medium_calorie
0,rating-5,weiblich,50-60 Jahre,Verheiratet,low_calorie,205,0,1,0
1,rating-4,weiblich,<30 Jahre,Vergeben,low_calorie,205,0,1,0
2,rating-5,weiblich,<30 Jahre,Vergeben,low_calorie,205,0,1,0
3,rating-3,weiblich,40-50 Jahre,Verheiratet,low_calorie,205,0,1,0
4,rating-5,weiblich,30-40 Jahre,Vergeben,low_calorie,205,0,1,0
...,...,...,...,...,...,...,...,...,...
82363,rating-1,männlich,<30 Jahre,Vergeben,medium_calorie,394,0,0,1
82364,rating-4,männlich,30-40 Jahre,Single,medium_calorie,300,0,0,1
82365,rating-4,männlich,30-40 Jahre,Single,high_calorie,559,1,0,0
82366,rating-5,männlich,50-60 Jahre,Vergeben,medium_calorie,332,0,0,1


In [83]:
data = upsampled_new[['sex', 'age_group', 'high_calorie', 'low_calorie', 'medium_calorie','marriage_status']]
target = ps.BinaryTarget('medium_calorie', True)
searchspace = ps.create_selectors(data, ignore=['high_calorie', 'low_calorie', 'medium_calorie'])
task = ps.SubgroupDiscoveryTask (
    data, 
    target, 
    searchspace, 
    result_set_size=5, 
    depth=2, 
    qf=ps.WRAccQF())
result = ps.BeamSearch().execute(task)
pd.set_option('max_colwidth',100)
print(result.to_dataframe())

    quality                                                description
0  0.003833                                     age_group=='60+ Jahre'
1  0.003353                 age_group=='60+ Jahre' AND sex=='männlich'
2  0.002635                                            sex=='männlich'
3  0.002375  age_group=='60+ Jahre' AND marriage_status=='Verheiratet'
4  0.002180         marriage_status=='Verheiratet' AND sex=='männlich'


- undersampling

In [84]:
# downsample majority
female_downsampled = resample(female,
                                replace = False, # sample without replacement
                                n_samples = len(male), # match minority n
                                random_state = 27) # reproducible results

# combine minority and downsampled majority
downsampled = pd.concat([female_downsampled, male])

# add the dummyset into the downsampled dataset
downsampled_dum = pd.get_dummies(downsampled['calorie_level'])
# downsampled_dum.reset_index(inplace = True)
# downsampled.reset_index(inplace = True)
downsampled_new = downsampled.join(downsampled_dum, how='left')
downsampled_new

,rating,sex,age_group,marriage_status,calorie_level,calorie_value,high_calorie,low_calorie,medium_calorie
20623,rating-5,weiblich,30-40 Jahre,Vergeben,medium_calorie,420,0,0,1
706,rating-4,weiblich,30-40 Jahre,Vergeben,high_calorie,2517,1,0,0
19310,rating-5,weiblich,40-50 Jahre,Verheiratet,medium_calorie,310,0,0,1
10530,rating-5,weiblich,40-50 Jahre,Single,low_calorie,287,0,1,0
19429,rating-5,weiblich,30-40 Jahre,Verheiratet,high_calorie,1118,1,0,0
...,...,...,...,...,...,...,...,...,...
48791,rating-4,männlich,50-60 Jahre,Verheiratet,high_calorie,511,1,0,0
48793,rating-5,männlich,40-50 Jahre,Verheiratet,high_calorie,511,1,0,0
48796,rating-5,männlich,60+ Jahre,Verheiratet,high_calorie,511,1,0,0
48800,rating-4,männlich,50-60 Jahre,Single,high_calorie,511,1,0,0


In [85]:
import pysubgroup as ps
data = downsampled_new[['sex', 'age_group', 'high_calorie', 'low_calorie', 'medium_calorie','marriage_status']]
target = ps.BinaryTarget('high_calorie', True)
searchspace = ps.create_selectors(data, ignore=['high_calorie', 'low_calorie', 'medium_calorie'])
task = ps.SubgroupDiscoveryTask (
    data, 
    target, 
    searchspace, 
    result_set_size=5, 
    depth=2, 
    qf=ps.WRAccQF())
result = ps.BeamSearch().execute(task)
pd.set_option('max_colwidth',100)
print(result.to_dataframe())

    quality                                               description
0  0.006648                                  age_group=='30-40 Jahre'
1  0.004331              age_group=='30-40 Jahre' AND sex=='männlich'
2  0.003175                                 marriage_status=='Single'
3  0.002458                                           sex=='männlich'
4  0.002379  age_group=='30-40 Jahre' AND marriage_status=='Vergeben'


#### Conclusion 

- the subgroup young single female seems to prefer the "low-calorie" recipes
- the subgroup married male seems to prefer the "high-calorie" recipes
- the subgroup divorced middle-age male seems to prefer the "medium-calorie" recipes

then our next step is to discover why differnt calorie-level is preferred by these particular groups

### Existing problems
- Here we remove all rows which have nan, does it make sense to use other fill nan methods? 
- the quality of downsampling is better than upsampling, but there are only 1040 rows left, is it enough?
- now the quality is around 0.01 is still low, how can we then to improve the quality

### Association rules among the recipe attributes

- the association with "ingredient" and "regions"

In [ ]:
countries = ['china', 'indien', 'deutschland','frankreich','grossbritannien','österreich','usaoderkanada','italien','spanien',
             'portugal', 'japen','schweiz','türkei', 'thailand', 'russland', 'großbritannien & irland', 'vietnam', 'korea',
            'australien', 'ägypten', 'marokko', 'niederlande']

In [ ]:
df_certain_countries = get_recipe_countries(countries, data)
df_certain_countries.reset_index(inplace = True)
df_certain_countries['label'].value_counts()

In [ ]:
# drop all the columns other than "ingredient" and "label" column
columns_drop = ['index', 'categorize', 'recipe_name', 'tags', 'avg_score', 'difficulty','rating_count', 'calorie', 
                'preparation_time','comment_user', 'recipe_url']
df_certain_countries = df_certain_countries.drop(columns_drop, axis = 1)

# preprocess the ingredient column
distinct_ingredients = ingredients_preprocess(df_certain_countries)

# One hot encoding of the ingredients
df_certain_countries['ingredient'] = df_certain_countries.ingredient.apply(convert_to_dict)
vectorizer = DictVectorizer(sparse=False)
new_df_countries = pd.DataFrame(data = vectorizer.fit_transform(df_certain_countries.ingredient.tolist()), columns = distinct_ingredients)
new_df_countries['label'] = df_certain_countries.label

# dummy for the label column
new_df_countries = pd.get_dummies(new_df_countries)

In [ ]:
# apply 
new_df_countries

In [ ]:
test = new_df_countries.iloc[:,]
test = test.iloc[:4]

In [ ]:
country_label = list(new_df_countries.iloc[:,-19:].columns)
#contry_label.remove('label_deutschland')

# record the start time
time1 = time.time()
        
# apply subgroup discovery 
target = ps.BinaryTarget ('label_usaoderkanada', True)
searchspace = ps.create_selectors(test, ignore = country_label)
task = ps.SubgroupDiscoveryTask (
    data, 
    target, 
    searchspace, 
    result_set_size=5, 
    depth=20, 
    qf=ps.WRAccQF())
result = ps.BeamSearch().execute(task)

# record the end time
time2 = time.time()
time_diff = (time2-time1)/60
print('it took ' + str(time_diff) + 'miniutes to execute the subgroup disc')